# Project 9: Support Vector Machines

## Goal
Understand margin maximization and kernel methods.

## Learning Objectives
- What is the margin?
- Support vectors and decision boundary
- Kernel trick and RBF kernel
- C parameter (regularization)

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import make_classification, make_circles
import sys
sys.path.append('/Users/mark/git/learning-ml-to-llm')

print("Ready to explore SVMs!")